In [42]:
from datetime import datetime
import requests
import json

In [47]:
f = open('issues.json', encoding="utf8")
issues = json.load(f)

In [48]:
f = open('commits.json', encoding="utf8")
commits = json.load(f)

In [49]:
print(commits[0]["commit"])

{'author': {'name': 'Doomsdayrs', 'email': '38189170+Doomsdayrs@users.noreply.github.com', 'date': '2022-10-03T01:07:57Z'}, 'committer': {'name': 'GitHub', 'email': 'noreply@github.com', 'date': '2022-10-03T01:07:57Z'}, 'message': 'Update update.json', 'tree': {'sha': 'ab1bea32cc167f377e7328e8e9829963cbfcf5a7', 'url': 'https://api.github.com/repos/shosetsuorg/shosetsu/git/trees/ab1bea32cc167f377e7328e8e9829963cbfcf5a7'}, 'url': 'https://api.github.com/repos/shosetsuorg/shosetsu/git/commits/692ab6ea892c6e03aa6d1530f460ceb873a1a029', 'comment_count': 0, 'verification': {'verified': True, 'reason': 'valid', 'signature': '-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQBQJjOjXtCRBK7hj4Ov3rIwAA/mkIADUa5Wey6d5tWl+ynAh+dOyS\nyh3YvoVEHXAXvA+C4cHGXVIx+SMhj3q8JRapaVSNM5UstmG89R8PqhBsSnvLybVP\nMf5svasFJACQVOOP6hy/6bJmZv3BI4AsySnTQskUi+TfO6gR1/+8gl9hc10Wbtm+\nPrTIypZQ3uV98u0xiSY+XQispfDXNhbFuIOZB8lh0Xc7T07c+MdY02KYrDpkJHeU\n31PQiJ4kjNAPFvuZfGeFWQWw+D5wtPOnrGAoU/w32lbioYFAlSsDNDxtEowot5t3\nzFsjpksr6U+

In [50]:
issue_commit_map = {}

for issue in issues:
    issue_created_at = issue["created_at"]
    issue_closed_at = issue["closed_at"]
    issue_created_at = datetime.fromisoformat(issue_created_at.replace('Z', '+00:00'))
    issue_closed_at = datetime.fromisoformat(issue_closed_at.replace('Z', '+00:00'))
    issue_commit_map[issue["id"]] = {"issue_title": issue["title"], "commit_messages":[]}
    for commit in commits:
    # print("created_at",obj["created_at"], "\nclosed_at", obj["closed_at"])
        commit_made_at = commit["commit"]["committer"]["date"]
        commit_made_at = datetime.fromisoformat(commit_made_at.replace('Z', '+00:00'))
        # print(commit_made_at)
        if commit_made_at>=issue_created_at and commit_made_at<=issue_closed_at:
            # new_item = {"issue_id": issue["id"], "commit_message":commit["commit"]["message"]}
            issue_commit_map[issue["id"]]["commit_messages"].append(commit["commit"]["message"])

# print(issue_commit_map)

issue_commit_map_list = list(issue_commit_map.values())
print(len(issue_commit_map_list))

100


In [51]:
from openai import OpenAI

v1 = "sk-5GmSxGb0TLx6jzpzhFbhT3B"
v2 = "lbkFJm3OoryQONwklze43ygTB"
OPENAI_API_KEY = v1+v2


client = OpenAI(api_key=OPENAI_API_KEY)


In [55]:
issue_passed_to_OPENAI = issue_commit_map_list[3]["issue_title"]
string2 = issue_commit_map_list[3]["commit_messages"]
string2 = ", ".join(string2)
string2 = string2.split('\n')
string2 = " ".join(string2)
string2 = string2.split('\t')
commit_messages_passed_to_OPENAI = " ".join(string2)

print("issue", issue_passed_to_OPENAI)
print("commtis", commit_messages_passed_to_OPENAI)

prompt_sent_to_OPENAI = "issue: "+ issue_passed_to_OPENAI + "\ncommit: "+ commit_messages_passed_to_OPENAI
print("prompt\n",prompt_sent_to_OPENAI)


issue Fullscreen Settings for Reader
commtis Fix padding, Fix enabled parameter on switch settings, MatchFullscreen is false by default, Address comment, Match fullscreen to focus option, Add fullscreen option, Clean up  Handle various build warnings and removing older classes, Update dependencies, Fix animation stutter, Re-add elevation and remove top content padding, Fix cover ContentScale, Fix expanded cover having a ratio and being rounded, NovelCard fixes, forced center alignment and outside padding, Fix scrolling filters, Add unread badge click, Licenses, Use a standard cover ratio, UI Cleanup and improvements
prompt
 issue: Fullscreen Settings for Reader
commit: Fix padding, Fix enabled parameter on switch settings, MatchFullscreen is false by default, Address comment, Match fullscreen to focus option, Add fullscreen option, Clean up  Handle various build warnings and removing older classes, Update dependencies, Fix animation stutter, Re-add elevation and remove top content padd

In [57]:
# #USE only when you need only percentage in the answer
# command_to_OPENAI = "You will receive two objects. One is an issue from gitHub. The other is a list of commit messages from gitHub. Now you have to give the rating out of 100, based on how closely related the issue and the commit are. If there is no commit message given , then give rating as 0/100. And give warning. If issue is not given do the same. Rating also should depend upon how closely they are refering to the same thing. There might be some commits which are not related to the issue, but try to ignore them and see if the commits which can be related are solving the issue or not. Do not give any explanation or any words. Just use a percentage to specify how close they are.."

##USE when you also need an explanation for it.
command_to_OPENAI_to_check_similarity = "You will receive two objects. One is an issue from gitHub. The other is a list of commit messages from gitHub. Now you have to give the rating out of 100, based on how closely related the issue and the commit are. If there is no commit message given , then give rating as 0/100. And give warning. If issue is not given do the same. Rating also should depend upon how closely they are refering to the same thing. There might be some commits which are not related to the issue, but try to ignore them and see if the commits which can be related are solving the issue or not. Also explain why do you think so? \n For example : issue: issue Fullscreen Settings for Reader \ncommit: Release debug update for r2417, Direct disclaimer to shosetsu website, Prevent concurrent modifies in CatalogViewModel via sync & copy, Add privacy policy button to about, Add more logs to RestoreBackupWorker.kt, Temp fix for backup restore with external repos, Release debug update for r2411, Fix padding, Fix enabled parameter on switch settings, MatchFullscreen is false by default, Address comment, Match fullscreen to focus option, Add fullscreen option, Clean up  Handle various build warnings and removing older classes, Update dependencies, Fix animation stutter, Re-add elevation and remove top content padding, Fix cover ContentScale, Fix expanded cover having a ratio and being rounded, NovelCard fixes, forced center alignment and outside padding, Fix scrolling filters, Add unread badge click, Licenses, Use a standard cover ratio, UI Cleanup and improvements.. \nFor this you can give a rating of 60 because there are commits which are not related to the issue but things like fix padding and MathcFullScreen are little matching. \nAnother example: issue: Move app package to app.shosetsu.android\n commit: Temp fix for backup restore with external repos.. \n For this you can give a rating of below 5 because the issue and commit are completely unrealted here..\nAnother example: issue: Fullscreen Settings for Reader\n commit: Fix padding, Fix enabled parameter on switch settings, MatchFullscreen is false by default, Address comment, Match fullscreen to focus option, Add fullscreen option, Clean up  Handle various build warnings and removing older classes, Update dependencies, Fix animation stutter, Re-add elevation and remove top content padding, Fix cover ContentScale, Fix expanded cover having a ratio and being rounded, NovelCard fixes, forced center alignment and outside padding, Fix scrolling filters, Add unread badge click, Licenses, Use a standard cover ratio, UI Cleanup and improvements \n For this you can give a rating of 80 or even 85 because the issue and commit are completely related here.. All teh commit is talking about solving the issue\nConsider these as examples and references and give the output for next prompts. Do not give any suggestions, only give explanation briefly\n First give rating then give explanation"

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": command_to_OPENAI_to_check_similarity},
    {"role": "user", "content": prompt_sent_to_OPENAI}
  ]
)

print("Rating based on how closely related they are..\n",completion.choices[0].message.content)

# #USE only when you need only percentage in the answer
command_to_OPENAI_to_check_dissimilarity = "You will receive two objects. One is an issue from gitHub. The other is a list of commit messages from gitHub. Now you have to give the rating out of 100, based on how unrelated the issue and the commit are. If the commit can indirectly solve the issue then do not consider it for rating. If there is no commit message given , then give rating as undefined. And give warning. If issue is not given do the same. Give small explanation.."

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": command_to_OPENAI_to_check_dissimilarity},
    {"role": "user", "content": prompt_sent_to_OPENAI}
  ]
)

print("Rating based on how unrelated they are..\n",completion.choices[0].message.content)

Rating based on how closely related they are..
 Rating: 95/100

Explanation: The commit messages provided are closely related to the issue of implementing Fullscreen Settings for Reader. Each commit addresses different aspects of the Fullscreen Settings, such as fixing padding, enabling parameters, matching fullscreen to focus option, adding fullscreen option, fixing animation stutter, and cleaning up UI. The commits directly address the implementation of the Fullscreen Settings, making the connection between the issue and the commits very strong. The only reason it's not a perfect 100 is because some commits may be for general cleanup or improvements that are not directly linked to the Fullscreen Settings, but the majority of the commits are directly related.
Rating based on how unrelated they are..
 Rating: 40/100

Explanation: The commit messages include a variety of fixes, updates, and improvements to the codebase, but it is not clear how directly they address the specific issue of

In [33]:
print(completion.choices[0].message.content)

Rating: 8/10

Explanation: The commit messages focus on various UI fixes and improvements in the code base, which do not directly relate to changing the export backup dialog to be easily understood. This indicates a high level of unrelatedness between the issue and the commit messages.
